In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
import numpy as np
from keras.preprocessing import image
import matplotlib.pyplot as plt
%matplotlib inline

def load_images_from_path(path, label):
    images = []
    labels = []

    for file in os.listdir(path):
        # Check if the file is an image file
        if file.endswith(('.jpg', '.jpeg', '.png')):  # Add other image extensions if needed
            img = image.load_img(os.path.join(path, file), target_size=(224, 224, 3))
            images.append(image.img_to_array(img))
            labels.append((label))

    # Print a message if no images were found
    if not images:
        print(f"No images found in the directory: {path}")

    return images, labels

import os
import numpy as np
from keras.preprocessing import image
import matplotlib.pyplot as plt
import math
%matplotlib inline

# ... (rest of the code remains the same)

def show_images(images, grid_size=(4, 4)):  # Default grid size is 4x4
    """
    Displays images in a grid format.

    Args:
        images: A list of images (NumPy arrays).
        grid_size: A tuple (rows, cols) specifying the grid dimensions.
    """
    num_images = len(images)

    # Handle the case where there are no images
    if num_images == 0:
        print("No images to display.")
        return

    # Calculate grid dimensions based on the number of images and grid size
    num_rows = grid_size[0]
    num_cols = grid_size[1]
    num_images_to_display = min(num_images, num_rows * num_cols)

    # Create subplots with the calculated grid dimensions
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 20), subplot_kw={'xticks': [], 'yticks': []})

    # Flatten the axes array for easier iteration
    axes = axes.flatten()

    # Display images on subplots
    for i in range(num_images_to_display):
        axes[i].imshow(images[i] / 255)

    # Hide any extra subplots if there are fewer images than grid cells
    for i in range(num_images_to_display, num_rows * num_cols):
        axes[i].axis('off')

    plt.tight_layout()  # Adjust spacing between subplots
    plt.show()

x_train = []
y_train = []
x_test = []
y_test = []

In [7]:
images, labels = load_images_from_path('/content/drive/MyDrive/data/train/arctic_fox', 0)
x_train += images
y_train += labels

In [8]:
images, labels = load_images_from_path('/content/drive/MyDrive/data/train/polar_bear', 1)

x_train += images
y_train += labels

In [9]:
images, labels = load_images_from_path('/content/drive/MyDrive/data/train/walrus', 2)


x_train += images
y_train += labels

In [10]:
images, labels = load_images_from_path('/content/drive/MyDrive/data/test/arctic_fox', 0)

x_test += images
y_test += labels

In [11]:
images, labels = load_images_from_path('/content/drive/MyDrive/data/test/polar_bear', 1)

x_test += images
y_test += labels

In [12]:
images, labels = load_images_from_path('/content/drive/MyDrive/data/test/walrus', 2)

x_test += images
y_test += labels

In [13]:
from tensorflow.keras.utils import to_categorical

x_train = np.array(x_train) / 255
x_test = np.array(x_test) / 255

y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

In [14]:
# Stage 1: Define a function to create a CNN model with specific hyperparameters
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout # Import Dropout
from keras.models import Sequential
from keras.optimizers import Adam

def create_tuned_cnn(kernel_size, num_filters, learning_rate, dropout_rate, dense_units, activation):
    tuned_model = Sequential([
        Conv2D(num_filters, kernel_size, activation=activation, input_shape=x_train.shape[1:]),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(dense_units, activation=activation),
        Dropout(dropout_rate),
        Dense(y_train_encoded.shape[1], activation='softmax')  # Adjust output layer for number of classes
    ])
    optimizer = Adam(learning_rate=learning_rate)
    tuned_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return tuned_model

In [15]:
# Stage 2: Define ranges for hyperparameters
from random import randint, uniform, choice

num_trials = 5 # Number of random combinations to try

# Store results for each trial
results = []


In [16]:
# Stage 3: Train and evaluate models with random hyperparameters
for trial in range(num_trials):
    print(f"Trial {trial + 1}/{num_trials}")

    # Randomly select hyperparameters
    kernel_size = randint(2, 5)  # Random kernel size between 2 and 5
    num_filters = randint(16, 64)  # Random number of filters between 16 and 64
    learning_rate = round(uniform(0.0001, 0.01), 5)  # Random learning rate between 0.0001 and 0.01
    dropout_rate = round(uniform(0.2, 0.5), 2)  # Random dropout rate between 0.2 and 0.5
    dense_units = randint(64, 256)  # Random number of neurons in the dense layer
    activation = choice(['relu', 'tanh'])  # Random activation function

    print(f"Testing with kernel_size={kernel_size}, num_filters={num_filters}, learning_rate={learning_rate}, "
          f"dropout_rate={dropout_rate}, dense_units={dense_units}, activation={activation}")

    # Create and train the model
    tuned_model = create_tuned_cnn((kernel_size, kernel_size), num_filters, learning_rate, dropout_rate, dense_units, activation)
    history = tuned_model.fit(x_train, y_train_encoded, epochs=5, batch_size=randint(32, 128), validation_data=(x_test, y_test_encoded), verbose=0)

    # Evaluate the model
    y_pred = tuned_model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_test_encoded, axis=1)

    # Compute classification metrics
    from sklearn.metrics import classification_report
    report = classification_report(y_true, y_pred_classes, output_dict=True)
    accuracy = report['accuracy']
    precision = np.mean([v['precision'] for k, v in report.items() if k.isdigit()])
    recall = np.mean([v['recall'] for k, v in report.items() if k.isdigit()])
    f1_score = np.mean([v['f1-score'] for k, v in report.items() if k.isdigit()])

    # Store trial results
    results.append({
        'kernel_size': kernel_size,
        'num_filters': num_filters,
        'learning_rate': learning_rate,
        'dropout_rate': dropout_rate,
        'dense_units': dense_units,
        'activation': activation,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score
    })
    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1_score:.4f}")


Trial 1/5
Testing with kernel_size=4, num_filters=35, learning_rate=0.00017, dropout_rate=0.31, dense_units=155, activation=tanh


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 845ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/keras/src

Accuracy: 0.3333, Precision: 0.1111, Recall: 0.3333, F1-Score: 0.1667
Trial 2/5
Testing with kernel_size=2, num_filters=47, learning_rate=0.00618, dropout_rate=0.23, dense_units=152, activation=relu
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 529ms/step
Accuracy: 0.3667, Precision: 0.2404, Recall: 0.3667, F1-Score: 0.2525
Trial 3/5
Testing with kernel_size=4, num_filters=28, learning_rate=0.00053, dropout_rate=0.3, dense_units=169, activation=tanh


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/keras/src

Accuracy: 0.3333, Precision: 0.1111, Recall: 0.3333, F1-Score: 0.1667
Trial 4/5
Testing with kernel_size=5, num_filters=26, learning_rate=0.00462, dropout_rate=0.34, dense_units=146, activation=tanh
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 757ms/step
Accuracy: 0.3333, Precision: 0.1111, Recall: 0.3333, F1-Score: 0.1667
Trial 5/5
Testing with kernel_size=3, num_filters=34, learning_rate=0.00942, dropout_rate=0.42, dense_units=73, activation=tanh


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/keras/src

4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 683ms/step
Accuracy: 0.3333, Precision: 0.1111, Recall: 0.3333, F1-Score: 0.1667


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
# Stage 4: Find and display the best hyperparameter combination
best_result = max(results, key=lambda x: x['accuracy'])
# Stage 4: Find and display the best hyperparameter combination
best_result = max(results, key=lambda x: x['accuracy'])
print("\nBest Hyperparameters:")
print(best_result)



Best Hyperparameters:
{'kernel_size': 2, 'num_filters': 47, 'learning_rate': 0.00618, 'dropout_rate': 0.23, 'dense_units': 152, 'activation': 'relu', 'accuracy': 0.36666666666666664, 'precision': 0.2403628117913832, 'recall': 0.36666666666666664, 'f1_score': 0.25254039646843246}


In [19]:
# Extract best hyperparameters
best_hyperparams = max(results, key=lambda x: x['accuracy'])
print("\nRetraining model with best hyperparameters:")
print(best_hyperparams)

# Create and retrain model with best hyperparameters
final_model = create_tuned_cnn(
    kernel_size=(best_hyperparams['kernel_size'], best_hyperparams['kernel_size']),
    num_filters=best_hyperparams['num_filters'],
    learning_rate=best_hyperparams['learning_rate'],
    dropout_rate=best_hyperparams['dropout_rate'],
    dense_units=best_hyperparams['dense_units'],
    activation=best_hyperparams['activation']
)

# Train the final model
history = final_model.fit(
    x_train, y_train_encoded,
    epochs=10,  # Adjust epochs for full convergence
    batch_size=64,  # You can use an optimal batch size
    validation_data=(x_test, y_test_encoded),
    verbose=1
)

# Print the training accuracy after each epoch
for epoch in range(len(history.history['accuracy'])):
    print(f"Epoch {epoch + 1}/{len(history.history['accuracy'])} - "
          f"Accuracy: {history.history['accuracy'][epoch]:.4f} "
          f"Loss: {history.history['loss'][epoch]:.4f}")

# Evaluate the final model on the test set
y_pred = final_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test_encoded, axis=1)

# Compute and compare metrics
from sklearn.metrics import classification_report
final_report = classification_report(y_true, y_pred_classes, output_dict=True)

# Print final model performance with training accuracy
final_training_accuracy = history.history['accuracy'][-1]  # Last training accuracy

print("\nFinal Model Performance:")
accuracy = final_report['accuracy']
precision = np.mean([v['precision'] for k, v in final_report.items() if k.isdigit()])
recall = np.mean([v['recall'] for k, v in final_report.items() if k.isdigit()])
f1_score = np.mean([v['f1-score'] for k, v in final_report.items() if k.isdigit()])

print(f"Training Accuracy: {final_training_accuracy:.4f}")  # Print training accuracy
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")



Retraining model with best hyperparameters:
{'kernel_size': 2, 'num_filters': 47, 'learning_rate': 0.00618, 'dropout_rate': 0.23, 'dense_units': 152, 'activation': 'relu', 'accuracy': 0.36666666666666664, 'precision': 0.2403628117913832, 'recall': 0.36666666666666664, 'f1_score': 0.25254039646843246}
Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 39s 6s/step - accuracy: 0.3797 - loss: 244.3294 - val_accuracy: 0.3333 - val_loss: 88.0565
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 35s 6s/step - accuracy: 0.2945 - loss: 57.1998 - val_accuracy: 0.4833 - val_loss: 2.8128
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 42s 6s/step - accuracy: 0.4408 - loss: 2.3317 - val_accuracy: 0.3667 - val_loss: 1.0719
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 38s 5s/step - accuracy: 0.6188 - loss: 0.8285 - val_accuracy: 0.5083 - val_loss: 0.9938
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 43s 6s/step - accuracy: 0.8177 - loss: 0.7397 - val_accuracy: 0.5583 - val_loss: 1.0342
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 41s 5s/step - accuracy: 0.8891 - loss